# 7.01 Lesson 1 key concepts
## Reading the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)


In [ ]:
data = pd.read_csv('learningSet.csv')

In [ ]:
print(data.shape)
data.head(10)

In [ ]:
data['TARGET_B'].value_counts() # did they make a donation or not
# whether or not they donate. binary.
# record of everyone send mail. 

In [ ]:
data['TARGET_D'].describe() # mean value of donation 0.79

# amount of money that people donate

In [ ]:
95412 * (0.793 - 0.68) # (mean donation for all people minus cost of mailing piece) * number of mailing pieces


In [ ]:
data[data['TARGET_B']>0]['TARGET_D'].describe() # people who have donated
# minimum and maximum donations 1 and 200

In [ ]:
4843 * (15.62 - 0.68) # (mean donation for all people who made a donation minus cost of mailing piece) * number of mailing pieces


To solve this problem we will first build a classification model to predict who will more likely respond and then for those respondents, we will build a regression model to predict the donation amount.

Then we can use the cost matrix to calculate the total benefit from the donations

Some of the challenges with the dataset are as follows:

Large number of features: The data set has over 450 features. Hence selecting the right features for the model is very critical and at the same time it is not easy as the same traditional ways of removing features is not effective given the large number of features. Apart from feature selection, feature extraction (creating your own features using the existing features) is also not easy in this case.
Sparsity of the dataset: There are a lot of features with a large number of null values.
Data imbalance: For developing a classification, there is a huge imbalance in the training dataset with only approximately 5000 values for one category as compared to cover 95,000 instances for the other category.

In [ ]:
# high number of features seems better. but there is a mathematical limit.

# this is an imbalanced dataset.

# 7.01 Lesson 2 key concepts
## Review data cleaning process

There are a lot of columns that have a very high percentage of null values. It is a highly sparse dataset. 
We can decide on a threshold and then remove those variables. There is no rule of thumb to decide on this threshold value. 
Sometimes it can as low as 25%-30%. And sometimes in some data sets you might find that even though there are more than 50% missing values in a column, you might have to include that variable in your analysis. 
A lot of it depends on the business context as well. In this case we will take this threshold to be 25% and then check the definitions of the columns filtered, to see if there is any column that we might want to keep.

In [ ]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index() # fraction of null values
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0].head(60)


In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0].tail(32)

In [ ]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']

In [ ]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

From the list above that includes the columns that have over 25% null values, you discussed with your manager you were told that the following columns are important -> wealth1, wealth2
We will remove these variables from the above list 
RDATE3, RAMNT_3 are important but they have too many null values 

# 7.01 Activity 2
## Remove columns with null values greater than a specified threshold

In [ ]:
drop_columns_list.remove('WEALTH1')

In [ ]:
drop_columns_list.remove('WEALTH2')

In [ ]:
len(drop_columns_list)

In [ ]:
# for i in ['WEALTH1','WEALTH2']:
#     drop_columns_list.remove(i)
# print(drop_columns_list)

In [ ]:
data = data.drop(columns=drop_columns_list)
# data = data.drop(drop_columns_list, axis=1)
data

## More data cleaning

In [ ]:
data.head()

We can see that there are a lot of columns that have blank spaces which represent no value in this case. 
They were not identified as null values by python as they are empty spaces that are read as character values by 
python. We will replace those values by NaNs and repeat the analysis 

Before we do that we will replace the blank values from the column "MAILCODE" by "A" which would mean the address is okay (pl check the definition of the variable in the description)

In [ ]:
data['MAILCODE'].value_counts()

In [ ]:
data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x.replace(" ", "A"))

# replace all space values with A

# FOR PRACTICAL REASONS, WE WANT TO KNOW WHETHER MAIL ADDRESS IS CORRECT OR NOT 


In [ ]:
# Now we can replace the rest space characters in other columns with np.NaN


In [ ]:
data = data.apply(lambda x: x.replace(" ", np.NaN))

In [ ]:
data.head()

# 7.01 Activity 3
## Remove columns with null values greater than a specified threshold

We would again repeat the same exercise as the last time. We will discuss it with the team, manager, and/or other stakeholders to see which columns we need to retain here. 

Like last time we will keep the following -> wealth1, wealth2 along with these VETERANS, SOLIH

In [ ]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index() # fraction of null values
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

In [ ]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']

In [ ]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

In [ ]:
for i in ['WEALTH1','WEALTH2','VETERANS','SOLIH']:
    drop_columns_list.remove(i)
print(drop_columns_list)

In [ ]:
len(drop_columns_list)

In [ ]:
data = data.drop(columns=drop_columns_list)
# data = data.drop(drop_columns_list, axis=1)
data

# 7.01 Lesson 4 key concepts
## Separating Target, Numerical Features, and Categorical Features

Since we have a huge number of features, it would be easier to work independently on numerical features and categorical features.

For the target variables, for now we will retain them both together. But later, we will build a classification model first where we would need the column TARGET_B only.

In [ ]:
Y = data[['TARGET_B', 'TARGET_D']]
Y.head()

# target that consists of two columns. upper case Y

In [ ]:
numerical = data.select_dtypes(np.number)
numerical = numerical.drop(columns = ['TARGET_B', 'TARGET_D'])
numerical.head()

In [ ]:
numerical.shape

In [ ]:
categorical = data.select_dtypes(object)
categorical.head()

In [ ]:
categorical.shape

## Working with categorical columns

We will work with the categorical features first. Look at the columns one by one. Some of the operations which we will perform are:

- Replace null values with the most occurring categories
- Reduce the number of categories in a column by grouping

It is important to note that some columns are defined by python as categorical/object types. There might be other columns defined as numerical that we want as categorical. We will look them later when we were working on numerical types.

In [ ]:
categorical.isna().sum()/len(categorical)

## Categorical Variables

Here we will try to reduce the number of categories. An ideal way would have been to group the states into 
regions. But in this case we will group all the states with counts less than 2500 into one category "other"

In [ ]:
df = pd.DataFrame(categorical['STATE'].value_counts()).reset_index()
df

# count: number of donations in each state


In [ ]:
df.columns = ['state', 'count']
other_states = list(df[df['count']<2500]['state']) # count is less than 2500
# other_states
def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x
    
categorical['STATE'] = list(map(clean_state, categorical['STATE']))


In [ ]:
categorical['STATE'].value_counts()

# 7.01 Lab | Revisiting Machine Learning Case Study

Complete the following steps on the categorical columns in the dataset:

Check for null values in all the columns

Exclude the following variables by looking at the definitions. Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

OSOURCE - symbol definitions not provided, too many categories
ZIP CODE - we are including state already
Identify columns that over 85% missing values

Remove those columns from the dataframe

Reduce the number of categories in the column GENDER. The column should only have either "M" for males, "F" for females, and "other" for all the rest

Note that there are a few null values in the column. We will first replace those null values using the code below:

## Check for null values in all the columns

In [ ]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index() # fraction of null values
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0].head(50)

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0].tail(50)

## Exclude the OSOURCE and ZIP variables

## Identify columns that over 85% missing values

In [ ]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.85]
columns_above_threshold['column_name']

In [ ]:
drop_list = list(columns_above_threshold['column_name'])
print(drop_list)

In [ ]:
categorical = data.select_dtypes(object)
categorical.head()

In [ ]:
for i in ['OSOURCE','ZIP']:
    drop_list.remove(i)
print(drop_list)

# There is error here. Because drop_list list does not have 'OSOURCE','ZIP' columns.
# That's why we can not drop (exclude) them from the list.

In [ ]:
data = data.drop(['OSOURCE', 'ZIP'], axis=1)

## Remove those columns from the dataframe

In [ ]:
data = data.drop(drop_list,axis=1)
data

## Reduce the number of categories in the column GENDER.

In [ ]:
df = pd.DataFrame(data['GENDER'].value_counts()).reset_index()
df

In [ ]:
print(data['GENDER'].value_counts())

In [ ]:
data['GENDER'].dtypes

In [ ]:
data['GENDER'].isna().sum()

In [ ]:
print(data['GENDER'].value_counts())
data['GENDER'] = data['GENDER'].fillna('F')

In [ ]:
df.columns = ['gender', 'count']
other_genders = list(df[df['count']<35000]['gender'])

def clean_gender(x):
    if x in other_genders:
        return 'other'
    else:
        return x
    
data['GENDER'] = list(map(clean_gender, data['GENDER']))


In [ ]:
data['GENDER'].value_counts()

In [ ]:
data['GENDER'].isna().sum()

# 7.02

We will work on the column DOMAIN. Let's replace the null values with the category that is represented the most and then we will split the data into two columns DOMAIN_A and DOMAIN_B.

- DOMAIN_A will consist of the first character from the DOMAIN
- DOMAIN_B will consist of the second character from the DOMAIN

Then we will drop the original column DOMAIN

In [ ]:
categorical['DOMAIN'].value_counts(dropna=False)

In [ ]:
categorical['DOMAIN'] = categorical['DOMAIN'].fillna('R2')

In [ ]:
categorical['DOMAIN_A'] = list(map(lambda x: x[0], categorical['DOMAIN'])) # first character

categorical['DOMAIN_B'] = list(map(lambda x: x[1], categorical['DOMAIN'])) # second character


In [ ]:
categorical.DOMAIN_A.value_counts()

In [ ]:
categorical = categorical.drop(columns=['DOMAIN'])

In [ ]:
categorical.head()

### Activity 2.1

In [ ]:
drop_list = []

In [ ]:
categorical['MDMAUD'].value_counts(dropna=False)

# major donor matrix code. since most of the data is unknown here we will drop this column.
#  'X' indicates that the donor is not a major donor.
# if xxxx we dont have any real values.

In [ ]:
categorical['MAILCODE'].value_counts(dropna=False)
# drop mailcode
# this column does not fit with the purpose of analysis who is more likely to donate
# this column is more administrative thing.

In [ ]:
categorical['NOEXCH'].value_counts(dropna=False).index
# categorical['NOEXCH'].value_counts(dropna=False)

# there are so many zeros. this column does not give us many information. drop this column.
# zero as number and as string
# NOEXCH` | Do Not Exchange Flag (For list rental) <br /> _=can be exchanged <br /> X = do not exchange

In [ ]:
#End Activity

## MDMAUD_XXX are also like MDMAUD



In [ ]:
categorical['MDMAUD_R'].value_counts()

In [ ]:
drop_list = drop_list + ['MDMAUD_R', 'MDMAUD_F','MDMAUD_A']

## Replacing NULL values

In [ ]:
categorical['CLUSTER'].value_counts(dropna=False)

In [ ]:
categorical['CLUSTER'].sort_values().unique()

In [ ]:
sorted(categorical['CLUSTER'].sort_values().index)

In [ ]:
categorical['CLUSTER'] = categorical['CLUSTER'].fillna('40')

In [ ]:
categorical['HOMEOWNR'].value_counts(dropna=False)

In [ ]:
categorical['HOMEOWNR'] = categorical['HOMEOWNR'].fillna('U')

In [ ]:
categorical['GENDER'].value_counts(dropna=False)

In [ ]:
#discuss why this makes more sense than the alternatives
categorical['GENDER'] = categorical['GENDER'].fillna('F')

### Activity 2.2.1

In [ ]:
categorical['DATASRCE'].value_counts(dropna=False)

In [ ]:
categorical['GEOCODE2'].value_counts(dropna=False)

In [ ]:
categorical['DATASRCE'] = categorical['DATASRCE'].fillna('other')

# metromail, polk, vender. replace NaN with 4 or other


In [ ]:
categorical['GEOCODE2'] = categorical['GEOCODE2'].fillna('A')

# replace NaN with A since it is the most frequent value.

In [ ]:
categorical.head()

## Removing columns with similar information

In [ ]:
categorical['RFA_6'].value_counts()

In [ ]:
# We will keep the column RFA_2X. We will delete rest of the columns 

for col_name in categorical.columns:
    if "RFA" in col_name:
        drop_list.append(col_name)       

In [ ]:
drop_list.remove('RFA_2R')
drop_list.remove('RFA_2A')
drop_list

In [ ]:
categorical = categorical.drop(columns=drop_list)
categorical.head()

In [ ]:
categorical.isna().sum()

# Working with numerical columns 

In [ ]:
numerical.head()

In [ ]:
numerical.shape

In [ ]:
df = pd.DataFrame(numerical.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

### Activity 2.2.2

Remove the columns starting with ADATE_.
We are assuming that the date when the previous mail was done is not significant in the respondents decision to give donation.
They may or may not even remember when they received the mail in the previous years. 
And for the column ADATE_2, check the values in the column. 
If the values are pretty much the same, then remove this column as well.

In [ ]:
drop_list = []
for col_name in numerical.columns:
    if 'ADATE' in col_name:
        drop_list.append(col_name)

In [ ]:
numerical['ADATE_4'].value_counts()

### Filling NULL values with plots

In [ ]:
numerical['AGE'] = numerical["AGE"].fillna(np.mean(numerical['AGE']))


In [ ]:
sns.distplot(numerical['AGE'])
plt.show()
#probably would be a bit better to use a predictive method here but this works

In [ ]:
numerical['AGE'].value_counts()

In [ ]:
# sns.distplot(numerical['INCOME'])  # this will not work as there are NaNs in the column
# sns.distplot(numerical[numerical['INCOME'].isna()==False]['INCOME']) 
sns.distplot(numerical['INCOME'])
plt.show()

# 7 possible values: categorical values

In [ ]:
# looks like the variable is actually categorical. We can verify it by using value_counts()

print(numerical['INCOME'].value_counts(dropna=False))
numerical['INCOME'] = numerical['INCOME'].astype(object)


In [ ]:
numerical.dtypes

In [ ]:
numerical['INCOME']

In [ ]:
# numerical.INCOME = np.where(numerical['INCOME']=='5.0',5.0,numerical['INCOME'])

# string 5.0 and numerical 5.0: 

In [ ]:
numerical['INCOME'] = numerical['INCOME'].fillna(5.0) 

# Replacing the null values with the most represented category
# EARLIER WE DID THIS BY PREDICTING income FROM a number of other numeric fields


In [ ]:
numerical['INCOME'].value_counts()

In [ ]:
numerical['INCOME'].dtypes

In [ ]:
sns.distplot(numerical[numerical['CLUSTER2'].isna()==False]['CLUSTER2']) 
plt.show()

In [ ]:
numerical['CLUSTER2'].value_counts(dropna=False)

In [ ]:
np.mean(numerical['CLUSTER2'])

In [ ]:
numerical['CLUSTER2'] = numerical['CLUSTER2'].fillna(np.ceil(np.mean(numerical['CLUSTER2'])))


In [ ]:
np.ceil(np.mean(numerical['CLUSTER2']))

In [ ]:
sns.distplot(numerical['CLUSTER2']) 
plt.show()

### Activity 2.3

Check if there are any other null values in the numerical data.
Clean the columns WEALTH2 and TIMELAG. Use appropriate method to fill the null values in these columns.

In [ ]:
numerical[numerical.columns[numerical.isna().any()]]

In [ ]:
numerical['WEALTH2'].value_counts(dropna=False)

In [ ]:
numerical['WEALTH2'].dtypes

In [ ]:
numerical['WEALTH2'].mean()

In [ ]:
numerical['WEALTH2'] = numerical['WEALTH2'].fillna(5.0)

# replace NaNs with the closest value to mean.

In [ ]:
numerical['WEALTH2'].value_counts(dropna=False)

In [ ]:
numerical['TIMELAG'].value_counts(dropna=False)

In [ ]:
numerical['TIMELAG'].dtypes

In [ ]:
numerical['TIMELAG'].median()

In [ ]:
numerical['TIMELAG'] = numerical['TIMELAG'].fillna(6.0)
# median value to replace because there are so many outliers in this column.


In [ ]:
numerical['TIMELAG'].value_counts(dropna=False)

In [ ]:
sns.distplot(numerical[numerical['WEALTH2'].isna()==False]['WEALTH2']) 
plt.show()

In [ ]:
numerical['WEALTH2'] = numerical['WEALTH2'].astype('object')
numerical['WEALTH2'] = numerical['WEALTH2'].fillna(5.0) # median

In [ ]:
sns.distplot(numerical[numerical['TIMELAG'].isna()==False]['TIMELAG']) 
plt.show()

In [ ]:
numerical['TIMELAG'] = numerical['TIMELAG'].fillna(np.ceil(np.mean(numerical['TIMELAG'])))


In [ ]:
sns.distplot(numerical['TIMELAG']) 
plt.show()

# Lab 2

# Lab | Feature engineering

Here we will work on cleaning some of the other columns in the dataset using the techniques that we used before in the lessons.

Check for null values in the numerical columns.
Use appropriate methods to clean the columns GEOCODE2, WEALTH1, ADI, DMA,and MSA.
Use appropriate EDA technique where ever necessary.

In [ ]:
numerical = data.select_dtypes(np.number) # numerical dataframe of data

In [ ]:
# fraction of null values in columns of numerical dataframe
nulls_percent_df = pd.DataFrame(numerical.isna().sum()/len(numerical)).reset_index() 
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0].head(50)

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0].tail(50)

## Wealth1 column

In [ ]:
sns.distplot(numerical['WEALTH1'])

In [ ]:
data['WEALTH1'].value_counts(dropna=False) # wealth rating

In [ ]:
data['WEALTH1'].dtypes

In [ ]:
data['WEALTH1'].mean()

In [ ]:
data['WEALTH1'] = data['WEALTH1'].fillna(5.0)

# replace NaNs with the closest value to mean.

In [ ]:
data['WEALTH1'].value_counts(dropna=False)

## ADI column

In [ ]:
sns.distplot(data['ADI'])

In [ ]:
data['ADI'].value_counts(dropna=False)

In [ ]:
data['ADI'].isna().sum()

In [ ]:
data['ADI'].dtypes

In [ ]:
data['ADI'].mean()

In [ ]:
data['ADI'] = data['ADI'].fillna(187)

# replace NaNs with the closest value to mean.

In [ ]:
data['ADI'].value_counts(dropna=False)

In [ ]:
data['ADI'].isna().sum()

## GEOCODE2 column

In [ ]:
data['GEOCODE2'].dtypes # GEOCODE2 column does not belong to numerical dataframe. 
# data types are object.

In [ ]:
data['GEOCODE2'].head(10) # Country size code

In [ ]:
data['GEOCODE2'].value_counts()

In [ ]:
data['GEOCODE2'].isna().sum() # There are 132 NaN values in GEOCODE2 column.

In [ ]:
# data['GEOCODE2'] = data['GEOCODE2'].replace(r'^\s*$', np.nan, regex=True)

data['GEOCODE2'] = data['GEOCODE2'].replace(r'^\s*$', 'A', regex=True)

# replace spaces with the most frequent value (A)

In [ ]:
data['GEOCODE2'].value_counts(dropna=False)

In [ ]:
data['GEOCODE2'] = data['GEOCODE2'].replace(np.nan, 'A', regex=True)
# replace spaces with the most frequent value (A)

In [ ]:
data['GEOCODE2'].value_counts(dropna=False)

In [ ]:
data['GEOCODE2'].isna().sum()

## DMA column

In [ ]:
sns.distplot(data['DMA'])

In [ ]:
data['DMA'].dtypes 

In [ ]:
data['DMA'].head(10) # Country size code

In [ ]:
data['DMA'].value_counts(dropna=False)

In [ ]:
data['DMA'].isna().sum() # There are 132 NaN values in DMA column.

In [ ]:
data['DMA'].mean()

In [ ]:
data['DMA'] = data['DMA'].replace(np.nan, 664, regex=True)

# replace NaNs with the closest value to the mean value of column

In [ ]:
data['DMA'].value_counts(dropna=False)

In [ ]:
data['DMA'].isna().sum()

## MSA column

In [ ]:
sns.distplot(data['MSA'])

In [ ]:
data['MSA'].dtypes 

In [ ]:
data['MSA'].head(10) # Country size code

In [ ]:
data['MSA'].value_counts(dropna=False)

In [ ]:
# There 21333 0.0 values. They can be replaced with median to get rid of the effect of outliers.

In [ ]:
data['MSA'].isna().sum() # There are 132 NaN values in MSA column.

In [ ]:
data['MSA'].median()

In [ ]:
data['MSA'] = data['MSA'].replace(0.0, 3350.0, regex=True)

# replace spaces with median

In [ ]:
data['MSA'].value_counts(dropna=False)

In [ ]:
data['MSA'].isnull().sum().sum()

In [ ]:
data['MSA'] = data['MSA'].replace(np.nan, 3350.0, regex=True)

In [ ]:
data['MSA'].isnull().sum().sum()

In [ ]:
data.to_csv('learningSet_partially_cleaned.csv')

# Wrapping up

In [ ]:
numerical = numerical.drop(columns=drop_list)

In [ ]:
df = pd.DataFrame(numerical.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

In [ ]:
numerical.to_csv('numerical7_02.csv')
categorical.to_csv('categorical7_02.csv')

In [ ]:
Y.to_csv('target7_02.csv')

In [ ]:
# alternative:
# numerical.to_pickle('numerical7_02.p')
# categorical.to_pickle('categorical7_02.p')
# Y.to_pickle('target7_02.p')